# Voice Cloning detection 

## Links of interest

### Data sets
- https://lionbridge.ai/datasets/best-speech-recognition-datasets-for-machine-learning/
- https://data.mendeley.com/datasets/k47yd3m28w/2
- https://keithito.com/LJ-Speech-Dataset/
- https://www.kaggle.com/charlesaverill/imagenet-voice
- https://www.kaggle.com/fabawi/augmented-extended-train-robots
- https://www.kaggle.com/jbuchner/synthetic-speech-commands-dataset
- https://www.kaggle.com/primaryobjects/voicegender

### Articles & Studies
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7058910/
- https://yorkspace.library.yorku.ca/xmlui/bitstream/handle/10315/36698/Reimao_Ricardo_AM_2019_Masters.pdf?sequence=2
- https://r9y9.github.io/deepvoice3_pytorch/

In [2]:
import numpy as np
import pandas as pd